# DataCivicLab – Variazioni nella raccolta differenziata e nei rifiuti urbani

## Obiettivo

Questo notebook fa parte del progetto pilota **DataCivicLab**.

Partendo dal dataset comunale ISPRA già pulito e aggregato (anni 2020–2023) da Alessio,
l’obiettivo è calcolare le **variazioni nel tempo** di:

- percentuale di raccolta differenziata
- rifiuti urbani totali
- rifiuti urbani pro capite

e preparare un **dataset finale pronto per la visualizzazione e la dashboard**.

L’analisi si concentra sulle **variazioni** più che sui livelli assoluti,
per rispondere direttamente alla domanda civica del progetto.

In [12]:
import pandas as pd

# Caricamento dataset aggregato e pulito (2020–2023)
df = pd.read_csv(
	"..\\data\\clean\\RU_Comunali_2020_2023_finale_aggregato_pulito.csv",
	sep=";",
	
)

df.head()


,istat_comune,comune,provincia,regione,popolazione,dato_riferito_a,altro_t_2020,altro_t_2021,altro_t_2022,altro_t_2023,...,totale_ru_t_2022,totale_ru_t_2023,verde_t_2020,verde_t_2021,verde_t_2022,verde_t_2023,vetro_t_2020,vetro_t_2021,vetro_t_2022,vetro_t_2023
0,1001001,AGLIE',Torino,Piemonte,2548.0,Comune,3.157,0.000,0.000,0.000,...,0.000,0.000,396.303,0.00,0.000,0.000,105.645,0.000,0.000,0.000
1,1001001,AGLIE',Torino,Piemonte,2549.0,Comune,0.000,1.868,0.000,0.000,...,0.000,0.000,0.000,226.06,0.000,0.000,0.000,98.494,0.000,0.000
2,1001001,AGLIE',Torino,Piemonte,2558.0,Comune,0.000,0.000,2.084,0.000,...,1415.399,0.000,0.000,0.00,284.359,0.000,0.000,0.000,96.739,0.000
3,1001001,AGLIE',Torino,Piemonte,2603.0,Comune,0.000,0.000,0.000,2.166,...,0.000,1330.664,0.000,0.00,0.000,171.517,0.000,0.000,0.000,96.183
4,1001002,AIRASCA,Torino,Piemonte,3569.0,Comune,0.025,0.000,0.000,0.000,...,0.000,0.000,144.097,0.00,0.000,0.000,109.330,0.000,0.000,0.000


In [21]:
# Selezione delle colonne rilevanti per il dashboard
df.columns

Index(['istat_comune', 'comune', 'provincia', 'regione', 'popolazione',
       'dato_riferito_a', 'altro_t_2020', 'altro_t_2021', 'altro_t_2022',
       'altro_t_2023', 'carta_e_cartone_t_2020', 'carta_e_cartone_t_2021',
       'carta_e_cartone_t_2022', 'carta_e_cartone_t_2023',
       'frazione_umida_t_2020', 'frazione_umida_t_2021',
       'frazione_umida_t_2022', 'frazione_umida_t_2023',
       'indifferenziato_t_2020', 'indifferenziato_t_2021',
       'indifferenziato_t_2022', 'indifferenziato_t_2023',
       'ingombranti_misti_t_2020', 'ingombranti_misti_t_2021',
       'ingombranti_misti_t_2022', 'ingombranti_misti_t_2023',
       'ingombranti_smaltimento_t_2020', 'ingombranti_smaltimento_t_2021',
       'ingombranti_smaltimento_t_2022', 'ingombranti_smaltimento_t_2023',
       'legno_t_2020', 'legno_t_2021', 'legno_t_2022', 'legno_t_2023',
       'metallo_t_2020', 'metallo_t_2021', 'metallo_t_2022', 'metallo_t_2023',
       'percentuale_rd_perc_2020', 'percentuale_rd_perc_2021',

In [23]:
#Calcolo rifiuti urbani pro capite per il 2020 e 2023
df["ru_pc_2020"] = df["totale_ru_t_2020"] * 1000 / df["popolazione"]
df["ru_pc_2023"] = df["totale_ru_t_2023"] * 1000 / df["popolazione"]

In [24]:
# Ordinamento per Comune e Anno
df["delta_rd_percent"] = (
    df["percentuale_rd_perc_2023"] - df["percentuale_rd_perc_2020"]
)

df["delta_ru_totali"] = (
    df["totale_ru_t_2023"] - df["totale_ru_t_2020"]
)

df["delta_ru_pro_capite"] = (
    df["ru_pc_2023"] - df["ru_pc_2020"]
)


In [25]:
df[[
    "delta_rd_percent",
    "delta_ru_totali",
    "delta_ru_pro_capite"
]].describe()



,delta_rd_percent,delta_ru_totali,delta_ru_pro_capite
count,30978.000000,3.097800e+04,30978.000000
mean,0.857375,1.057701e+01,1.236869
std,49.254223,1.637871e+04,592.097804
min,-100.000000,-1.529044e+06,-39681.456140
25%,0.000000,0.000000e+00,0.000000
50%,0.000000,0.000000e+00,0.000000
75%,0.225000,7.139500e+00,26.175643
max,99.920000,1.601781e+06,55712.147059


In [26]:
dashboard_df = df[
    [
        "comune",
        "delta_rd_percent",
        "delta_ru_totali",
        "delta_ru_pro_capite"
    ]
].copy()


In [27]:
dashboard_df.to_csv(
    "../data/clean/comuni_delta_2020_2023_dashboard.csv",
    index=False
)


In [29]:
# Esplorazione dei comuni con miglioramenti in RD e aumento dei rifiuti totali
dashboard_df[
    (dashboard_df["delta_rd_percent"] > 0) &
    (dashboard_df["delta_ru_totali"] > 0)
].head()


,comune,delta_rd_percent,delta_ru_totali,delta_ru_pro_capite
3,AGLIE',62.61,1330.664,511.203995
7,AIRASCA,72.24,1806.090,489.986435
11,ALA DI STURA,44.92,346.608,732.786469
12,ALBIANO D'IVREA,80.69,708.703,437.741198
17,ALMESE,77.32,3594.304,568.449154


In [30]:
dashboard_df[
    (dashboard_df["delta_rd_percent"] > 0) &
    (dashboard_df["delta_ru_totali"] > 0)
].shape


(7730, 4)

## Nota metodologica

Le variazioni sono calcolate come differenza tra il primo e l’ultimo anno del periodo (2020–2023),
così da sintetizzare l’andamento complessivo ed evitare rumore dovuto a oscillazioni annuali.
Gli anni intermedi (2021–2022) vengono utilizzati in fase di esplorazione e visualizzazione
per fornire contesto ai casi selezionati.